In [1]:
import pandas as pd
import numpy

In [2]:
data = pd.read_csv("dexcomData.csv")
data = data.sort_values(["Timestamp"])
full_data = pd.DataFrame(data={"Blood Glucose":data["Blood Glucose (mg/dL)"], "Timestamp":data["Timestamp"]})
full_data = full_data.set_index("Timestamp")
full_data.describe()

,Blood Glucose
count,73526.000000
mean,156.356704
std,65.425254
min,39.000000
25%,108.000000
50%,146.000000
75%,194.000000
max,401.000000


In [4]:
print(full_data.index.max())
print(full_data.index.min())

2018-10-27 21:27:07
2017-09-11 22:22:07


In [37]:
from dateutil import rrule
from datetime import datetime, timedelta

start = datetime.strptime(full_data.index.min(), "%Y-%m-%d %H:%M:%S")
end = datetime.strptime(full_data.index.max(), "%Y-%m-%d %H:%M:%S")
months = []

for dt in rrule.rrule(rrule.MONTHLY, dtstart=start, until=end):
    months.append(datetime.strftime(dt, "%Y-%m-%d %H:%M:%S")[:7])
    
print(months)

['2017-09', '2017-10', '2017-11', '2017-12', '2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06', '2018-07', '2018-08', '2018-09', '2018-10']


In [60]:
month_summary = pd.DataFrame()
month_summary[months[0]] = full_data.loc[full_data.index < months[1]].describe()["Blood Glucose"]
for month in range(1, len(months)-1 ):
    last_month = months[month - 1]
    current_month = months[month]
    next_month = months[month + 1]
    mask = (full_data.index > last_month) & (full_data.index < next_month)
    month_summary[current_month] = full_data.loc[mask].describe()["Blood Glucose"]
    last_month = month
month_summary[months[-1]] = full_data.loc[full_data.index > months[-1]].describe()["Blood Glucose"]
month_summary   

,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10
count,3904.000000,3948.000000,44.000000,6021.000000,13608.000000,10481.000000,9879.000000,12077.000000,13262.000000,15521.000000,11486.000000,12076.000000,14052.000000,7291.000000
mean,161.377818,161.015957,128.909091,152.246139,160.754924,161.248354,150.007794,157.488118,159.905369,153.418852,148.603779,144.914376,154.966268,162.666027
std,59.245200,59.248221,50.574292,64.243884,68.836560,67.205272,58.760331,65.738288,69.625434,67.678565,65.203752,61.409411,64.077299,63.672134
min,39.000000,39.000000,75.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,40.000000
25%,115.000000,115.000000,82.000000,100.000000,107.000000,110.000000,108.000000,112.000000,108.000000,101.000000,99.000000,99.000000,109.000000,118.000000
50%,156.500000,156.000000,114.500000,143.000000,150.000000,151.000000,141.000000,146.000000,147.000000,141.000000,139.000000,135.000000,144.000000,151.000000
75%,201.000000,201.000000,170.250000,195.000000,204.000000,203.000000,183.000000,190.000000,199.000000,195.000000,186.000000,178.000000,188.000000,196.000000
max,350.000000,350.000000,226.000000,370.000000,401.000000,401.000000,401.000000,401.000000,401.000000,401.000000,401.000000,401.000000,401.000000,401.000000


In [61]:
month_summary.to_csv("../data/monthlyStats.csv")